# GEN AI Workflow Notebook ✨✨

ClassifAI allows users to create vector databases from text datasets, search those vector datasets in an ad-hoc retrieval manner, and deploy this pipeline as a restAPI service using FastAPI to perform AI assisted classification tasks.


An recent emergent AI field is Retrieval Augmented Generation (RAG) where text generation models that traditionally respond to a user 'prompt', first retrieve relvant infomration from a vector database via adhoc retrieval processes, and use those serach results as context to generate an answer for the original user prompt.

#### This notebook shows how RAG can be implemented into our ClassifAI utilising its existing retrieval augmentation capabilities


## ClassifAIs existing retrieval setup

![Server_Image](files/servers.png)

#### The other modules of ClassifAI provide 3 core classes that work together to:

1. Vectorisers, to create embeddings,
2. Vectorstores, to create datgabsees of vectors and the ability to searc/query those database
3. RestAPI, to deploy a rest api service on a server to allow connections that search the created vector databases

In [ ]:
!uv pip install "classifai[gcp]"

## Setup

We need to first set up a traditional ClassifAI pipeline that can provide search/classification results for our generative model to use as context...


All of the following code is part of our standard ClassifAI setupm, and is a short demo of how you can create a semantic search classification system. <b>Check out our general_workflow_demo.ipynb notebook for a walkthrough of the content of these cells!</b>

In [ ]:
!gcloud auth application-default login

In [ ]:
from classifai.indexers import VectorStore
from classifai.vectorisers import GcpVectoriser

vectoriser = GcpVectoriser(
    project_id="<your-gcp-project-id>",
    model_name="text-embedding-004",
    task_type="CLASSIFICATION",
)


my_vector_store = VectorStore(
    file_name="./data/fake_soc_dataset.csv",  # demo csv file from the classifai package repo! (try some of our other DEMO/data datasets)
    data_type="csv",
    vectoriser=vectoriser,
    overwrite=True,
)

## We've set up our semantic search classification system

The following cell runs the 'search' method of the vectorstore, which will take a query for our created vectorstore and return the top k (n_results) most similar samples stored in the vectorstore. This is an example of our exisiting retrieval capabilities with the ClassifAI package. 

This retrieved set can then be used to make a final classification decision, by some method such as:
- automatically choosing the top ranked item, 
- a human in the loop physically looking at the retrieved candidates making the decision,
- by using a generative AI agent to assess the candidate lists and making a final decison... more on that later

In [ ]:
my_vector_store.search("dairy famer that sells milk", n_results=5)

## Creating a RAG Agent

With a way of getting context information for a query/prompt we can now instantiate a generative model


![Rag_Image](files/agent.png) 


In essence, the RAG agent has a .generate method which accepts a string query. It internally calls the associated vectorstore's search method using that query provided by the user. The original user query and the vectorstore results are used to generate a response.




To instantiate the object it takes:

In [ ]:
from classifai.generators import RagAgent_GCP

my_agent = RagAgent_GCP(
    project_id="<your-gcp-project-id>",
    model_name="gemini-2.5-flash",
    vectorStore=my_vector_store,
    task_type="classification",
)

#### Similar to the Vectorsiser it has a transform method -

this executes a vector store search, then uses that ranking and the user query to generate a response in line with the TASK_TYPE.

In the above case we are using the <b>"classification"</b> task type which tells the model [using a system prompt](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions) which guides our model in what to do with our retrieved ranking and user prompt/input. 

We also currently provide a <b>"summarization"</b> task type which uses a different system prompt we've built that instructs the model to provide a laymans summary of the content of the top k retrieved results. We won't show that in this demo though.

In [ ]:
my_agent.transform("apple farmer and other farming types")

## Thats it!

You can see in the above cell that my_agent.transform()  returns a 'text' entry and a 'ranking' entry. The 'text' entry contains a string "classification: X", where X will be replaced by the final classification decision that the agent decided was correct given the user query and retreived ranking.

Check out the general_worfklow notebook to see how VectorStore models AND Generative RAG models can be deployed. Both can be passed to the start_api function, to deploy these models in a restAPI.